# 🚀 E/L Hypothesis Testing Pipeline

Complete pipeline for testing H*: β_VF > 0 (Flexibility amplifies Vagueness effect on Later success)

## Pipeline Stages
1. **🏗️ BUILD**: Convert .dat to .parquet and consolidate E=1 cohort
2. **🧠 DEFINE**: Define E, L, V, F variables
3. **📊 PLOT1**: Sanity check each variable (E, L, V, F distributions)
4. **⚖️ TEST**: Test hypothesis and validate design
5. **📈 PLOT2**: Final EVF/LVF interaction plots

---

## E and L Definitions

**E (Short term survival)**:
- E is defined as funding amount of companies whose most recent financing was Early Stage VC (Series A) as of Dec 2021
- State-based definition (not event-based)
- NO Buyout/LBO, NO Merger/Acquisition

**L (Long term survival)**:
- Among companies with E=1, by year t, did they secure Series B+?
- t ∈ {2023, 2024, 2025}

---

In [ ]:
# Setup: Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import sys
import re

# Add modules to path
sys.path.insert(0, str(Path.cwd()))

from modules import models, plots_F_series
from modules.features import compute_vagueness_vectorized, classify_hardware_vectorized

# Configure display
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', 1000)

# Configure plotting
plt.rcParams['figure.figsize'] = (12, 6)
sns.set_style('whitegrid')

print("✅ Setup complete!")

: 

---
## Stage 0: 🔍 DATA EXPLORATION (나대용 지원용)

P1 "Vagueness as Preserved Options" 분석을 위한 데이터 구조 탐색.

**탐색 목표**:
1. NetCDF / Parquet 데이터의 전체 컬럼 구조
2. **TechReadiness proxy** 후보 열 확인 (patent, tech keywords 등)
3. Series B survival 변수 확인

---

In [ ]:
# =============================================================================
# 🔍 Stage 0: Data Exploration for P1 "Vagueness as Preserved Options"
# =============================================================================
# NetCDF 파일 기반 탐색 (터미널에서도 실행 가능)

import xarray as xr
from pathlib import Path

print("="*80)
print("🔍 P1 DATA EXPLORATION: TechReadiness Proxy 탐색 (NetCDF)")
print("="*80)

# 데이터 파일 경로
PROCESSED_DIR = Path("data/processed")

# 탐색할 NetCDF 파일들
NC_FILES = [
    "features_all.nc",
    "companies_21_23-24-25_transportation.nc",
    "companies_21_23-24-25.nc",
    "consolidated_companies.nc",
]

for fname in NC_FILES:
    fpath = PROCESSED_DIR / fname
    if fpath.exists():
        print(f"\n{'='*80}")
        print(f"📁 {fname}")
        print("="*80)
        try:
            ds = xr.open_dataset(fpath)
            
            # 기본 정보
            print(f"\n📐 DIMENSIONS: {dict(ds.dims)}")
            print(f"📍 COORDINATES: {list(ds.coords)}")
            
            # 모든 데이터 변수 출력
            print(f"\n📋 ALL DATA VARIABLES ({len(ds.data_vars)}):")
            for i, var in enumerate(ds.data_vars):
                dtype = ds[var].dtype
                dims = ds[var].dims
                print(f"   {i+1:3d}. {var:<40} | dims={dims} | dtype={dtype}")
            
            # TechReadiness proxy 후보 검색
            print(f"\n🔎 TECHREADINESS PROXY 후보:")
            tech_keywords = ['patent', 'tech', 'algorithm', 'proprietary', 'ip', 
                           'invention', 'license', 'r&d', 'research', 'hardware', 'software']
            found_tech = False
            for var in ds.data_vars:
                var_lower = var.lower()
                for kw in tech_keywords:
                    if kw in var_lower:
                        found_tech = True
                        arr = ds[var].values
                        print(f"   ✓ {var}")
                        try:
                            import numpy as np
                            if np.issubdtype(arr.dtype, np.number):
                                valid = arr[~np.isnan(arr)] if np.issubdtype(arr.dtype, np.floating) else arr
                                print(f"     Stats: mean={np.mean(valid):.3f}, std={np.std(valid):.3f}, unique={len(np.unique(valid))}")
                            else:
                                unique_vals = list(set(arr.flatten()))[:5]
                                print(f"     Sample values: {unique_vals}")
                        except:
                            pass
                        break
            if not found_tech:
                print("   (없음 - Text 기반 proxy 필요)")
            
            # Survival / Series B 관련 변수 검색
            print(f"\n🎯 SURVIVAL (Series B) 관련 변수:")
            survival_keywords = ['series', 'stage', 'later', 'funding', 'deal', 'growth', 'L_', 'survival']
            found_survival = False
            for var in ds.data_vars:
                var_lower = var.lower()
                for kw in survival_keywords:
                    if kw in var_lower:
                        found_survival = True
                        arr = ds[var].values
                        print(f"   ✓ {var}")
                        try:
                            import numpy as np
                            if np.issubdtype(arr.dtype, np.number):
                                valid = arr[~np.isnan(arr)] if np.issubdtype(arr.dtype, np.floating) else arr
                                print(f"     Stats: mean={np.mean(valid):.3f}, unique={len(np.unique(valid))}")
                        except:
                            pass
                        break
            if not found_survival:
                print("   (L_ 변수 없음 - DealType에서 파생 필요)")
            
            # Vagueness 관련 변수
            print(f"\n🌫️ VAGUENESS 관련 변수:")
            vague_keywords = ['vague', 'v_', 'z_v']
            for var in ds.data_vars:
                var_lower = var.lower()
                for kw in vague_keywords:
                    if kw in var_lower:
                        arr = ds[var].values
                        print(f"   ✓ {var}")
                        try:
                            import numpy as np
                            valid = arr[~np.isnan(arr)]
                            print(f"     Stats: mean={np.mean(valid):.3f}, std={np.std(valid):.3f}")
                        except:
                            pass
                        break
            
            # Attributes
            if ds.attrs:
                print(f"\n📝 ATTRIBUTES:")
                for k, v in ds.attrs.items():
                    print(f"   {k}: {v}")
            
            ds.close()
            
        except Exception as e:
            print(f"   ❌ Error: {e}")
    else:
        print(f"\n❌ {fname} - NOT FOUND")

print("\n" + "="*80)
print("✅ NetCDF 데이터 탐색 완료!")
print("="*80)
print("""
📋 나대용(Claude)에게 이 출력 전체를 복사해 주세요.
   - TechReadiness proxy 후보 확인
   - Series B survival 변수 확인  
   - Vagueness 변수 현황 확인
""")

---
## Stage 1: 🏗️ BUILD - Data Consolidation

Convert .dat files to .parquet and consolidate E=1 cohort with endpoint years.

**Steps**:
1. Convert .dat → .parquet (if needed)
2. Load consolidated E=1 cohort
3. Verify E=1 filtering worked correctly

In [ ]:
%%time
# Step 1: Check if consolidation needed
PROCESSED_DIR = Path("data/processed")
INPUT_FILE = PROCESSED_DIR / "companies_21_23-24-25.parquet"

if not INPUT_FILE.exists():
    print("❌ Consolidated file not found!")
    print(f"   Expected: {INPUT_FILE}")
    print("\nRun consolidation first:")
    print("   python scripts/consolidate_2021_cohort.py")
    raise FileNotFoundError(f"Missing: {INPUT_FILE}")

# Step 2: Load consolidated data
print(f"Loading consolidated E=1 cohort from: {INPUT_FILE.name}")
print("-" * 80)

df = pd.read_parquet(INPUT_FILE)

print(f"\n✅ BUILD Stage Complete!")
print(f"\n📊 Data Summary:")
print(f"   Rows: {len(df):,} (all E=1 companies)")
print(f"   Columns: {len(df.columns)}")
print(f"   Memory: {df.memory_usage(deep=True).sum() / 1e6:.1f} MB")

In [ ]:
# Step 3: Verify E=1 cohort purity
print("Verifying E=1 cohort purity...")
print("-" * 80)

# Pattern for E=1
PAT_E = re.compile(r"(?:\bEarly\s*Stage\s*VC\b|\bSeries\s*A(?:\b|[\s\-]?\d*)\b)", re.I)

# Check all companies have E=1
is_E = df['DealType_2021'].fillna('').apply(lambda x: bool(PAT_E.search(str(x))))
e_rate = is_E.mean()

print(f"\nE=1 rate: {is_E.sum():,} / {len(df):,} ({e_rate:.1%})")

if e_rate == 1.0:
    print("✅ PASS: All companies are E=1 (Early Stage VC)")
else:
    print(f"❌ FAIL: Found {(~is_E).sum():,} non-E=1 companies!")
    print("\nSample non-E companies:")
    display(df[~is_E][['CompanyID', 'CompanyName', 'DealType_2021']].head())

# Check available columns
print(f"\n📋 Available columns:")
print(f"   {list(df.columns)}")

---
## Stage 2: 🧠 DEFINE - Variable Definition

Define core variables:
- **E**: Early Stage VC status (all = 1 in this cohort)
- **L**: Later Stage VC / Series B+ achievement (2025)
- **V**: Vagueness score from Description/Keywords
- **F**: Flexibility (1 - is_hardware)

**CRITICAL**: E is a **mediator**, not a confounder. Do NOT include E in L regression.

In [ ]:
# Define E (Early funding amount - CONTINUOUS)
print("Defining E (Early Stage VC funding amount)...")
print("-" * 80)

# E is the funding AMOUNT (continuous), not binary status
if 'E_amount_2021' in df.columns:
    print("\n✓ Using E_amount_2021 (funding amount from raw data)")
    df['E'] = df['E_amount_2021']
    
    # Scale E for better regression (millions of USD)
    if df['E'].max() > 1e6:  # If values look like they're in USD
        df['E_scaled'] = df['E'] / 1e6  # Convert to millions
        print(f"   Scaled to millions: E_scaled = E / 1e6")
    else:
        df['E_scaled'] = df['E']
    
    print(f"\n💵 E (Early Funding Amount):")
    print(f"   Count: {df['E'].notna().sum():,}")
    print(f"   Mean:   ${df['E'].mean():,.0f}")
    print(f"   Median: ${df['E'].median():,.0f}")
    print(f"   Std:    ${df['E'].std():,.0f}")
    print(f"   Min:    ${df['E'].min():,.0f}")
    print(f"   Max:    ${df['E'].max():,.0f}")
else:
    print("\n⚠️  WARNING: E_amount_2021 not found in dataset!")
    print("   This column should contain LastFinancingSize from raw data")
    print(f"\n   Available columns: {list(df.columns)}")
    print(f"\n   To fix:")
    print(f"   1. Ensure raw data has 'LastFinancingSize' column")
    print(f"   2. Re-run: python scripts/consolidate_2021_cohort.py")
    print(f"\n   For now, using mock E values for demonstration...")
    
    # Mock E for demonstration (log-normal distribution typical for VC)
    np.random.seed(42)
    df['E'] = np.random.lognormal(mean=np.log(5e6), sigma=1.0, size=len(df))
    df['E_scaled'] = df['E'] / 1e6
    
    print(f"\n   Mock E statistics:")
    print(f"   Mean:   ${df['E'].mean():,.0f}")
    print(f"   Median: ${df['E'].median():,.0f}")

# Verify cohort filter: all companies should be in E=1 state (Early Stage VC)
is_E_state = df['DealType_2021'].fillna('').apply(
    lambda x: bool(PAT_E.search(str(x)))
)
e_state_rate = is_E_state.mean()

print(f"\n📋 E=1 Cohort Verification:")
print(f"   Companies in Early Stage VC state: {is_E_state.sum():,} ({e_state_rate:.1%})")

if e_state_rate < 1.0:
    print(f"   ⚠️  WARNING: Not all companies are in E=1 state!")
    print(f"   Expected: 100% (cohort is pre-filtered)")
else:
    print(f"   ✅ All companies in E=1 state (as expected)")

print(f"\n   Note: E (funding amount) is CONTINUOUS, E=1 state is binary filter")

In [ ]:
# Define L (Later Stage VC / Series B+)
print("Defining L (Later Stage VC / Series B+)...")
print("-" * 80)

# Pattern for L=1
PAT_L = re.compile(r"(?:\bLater\s*Stage\s*VC\b|\bSeries\s*[B-G](?:\b|[\s\-]?\d*)\b)", re.I)

# Define L for each year
for year in [2023, 2024, 2025]:
    col = f'DealType_{year}'
    if col in df.columns:
        df[f'L_{year}'] = df[col].fillna('').apply(
            lambda x: 1 if bool(PAT_L.search(str(x))) else 0
        )
    else:
        print(f"   ⚠️  {col} not found, skipping")

# Use 2025 as primary L
df['L'] = df['L_2025'] if 'L_2025' in df.columns else 0

print(f"\n💰 L (Later Stage) by year:")
for year in [2023, 2024, 2025]:
    if f'L_{year}' in df.columns:
        l_count = df[f'L_{year}'].sum()
        l_rate = df[f'L_{year}'].mean()
        print(f"   L_{year}: {l_count:,} ({l_rate:.1%})")

# Primary L (2025)
l_count = df['L'].sum()
l_rate = df['L'].mean()
print(f"\n   Primary L (2025): {l_count:,} ({l_rate:.1%})")

In [ ]:
# Define V (Vagueness)
print("Defining V (Vagueness)...")
print("-" * 80)

# Check if Description/Keywords available
has_desc = 'Description_2021' in df.columns
has_keywords = 'Keywords_2021' in df.columns

if has_desc and has_keywords:
    print("\n✓ Computing vagueness from Description/Keywords...")
    
    # Compute vagueness
    df['V'] = compute_vagueness_vectorized(
        df['Description_2021'].fillna(''),
        df['Keywords_2021'].fillna('')
    )
    
    # Z-score
    df['z_V'] = (df['V'] - df['V'].mean()) / df['V'].std()
    
    print(f"\n🤙 V (Vagueness):")
    print(f"   Count: {df['V'].notna().sum():,}")
    print(f"   Mean: {df['V'].mean():.2f}")
    print(f"   Std:  {df['V'].std():.2f}")
    print(f"   Min:  {df['V'].min():.2f}")
    print(f"   Max:  {df['V'].max():.2f}")
    
    if df['V'].std() < 5:
        print(f"\n⚠️  WARNING: Vagueness std ({df['V'].std():.2f}) is very small!")
        print(f"   Expected std: 15-25 for real venture data")
        print(f"   This may indicate data quality issues.")
        print(f"\n   Run diagnosis:")
        print(f"     python scripts/diagnose_vagueness_quality.py")
else:
    print(f"\n❌ Description/Keywords not found!")
    print(f"   Description_2021: {'✓' if has_desc else '✗'}")
    print(f"   Keywords_2021: {'✓' if has_keywords else '✗'}")
    print(f"\n   Using mock vagueness for demonstration")
    
    df['V'] = np.random.normal(50, 15, len(df))
    df['z_V'] = (df['V'] - df['V'].mean()) / df['V'].std()

In [ ]:
# Define F (Flexibility = 1 - is_hardware)
print("Defining F (Flexibility)...")
print("-" * 80)

# Infer F from CompanyName if not available
if 'F_flexibility' not in df.columns:
    print("\n✓ Inferring F from CompanyName keywords...")
    
    if 'CompanyName' in df.columns:
        sw_keywords = ['software', 'platform', 'cloud', 'saas', 'app', 'digital', 'analytics']
        sw_mask = pd.Series([False] * len(df))
        
        for kw in sw_keywords:
            sw_mask |= df['CompanyName'].fillna('').str.contains(kw, case=False)
        
        df['F_flexibility'] = sw_mask.astype(int)
    else:
        print("   ⚠️  CompanyName not found, using F=0 for all")
        df['F_flexibility'] = 0

f1_count = (df['F_flexibility'] == 1).sum()
f0_count = (df['F_flexibility'] == 0).sum()
f1_rate = f1_count / len(df)

print(f"\n💪 F (Flexibility):")
print(f"   F=1 (Flexible/SW): {f1_count:,} ({f1_rate:.1%})")
print(f"   F=0 (Rigid/HW):    {f0_count:,} ({(1-f1_rate):.1%})")

if f1_count == 0 or f0_count == 0:
    print(f"\n⚠️  WARNING: F has only one value!")
    print(f"   Interaction test will not be possible.")

In [ ]:
# Add controls (mock for demonstration)
print("Adding control variables...")
print("-" * 80)

# Mock controls
df['founding_cohort'] = 'cohort_1'
df['region'] = 'US'

print(f"\n✅ Control variables added (mock)")
print(f"   founding_cohort: all set to 'cohort_1'")
print(f"   region: all set to 'US'")
print(f"\n   Note: Replace with real founding year and HQ region for production")

---
## Stage 3: 📊 PLOT1 - Sanity Check

Visualize each variable to ensure they make sense.

**What to check**:
- E: Should be all 1 (by design)
- L: Should be 15-40% (reasonable progression rate)
- V: Should have mean ~50, std ~15-25 (if real data)
- F: Should have both 0 and 1 values

In [ ]:
# Plot E distribution (continuous)
fig, ax = plt.subplots(figsize=(10, 5))

# Use E_scaled for better visualization
E_col = 'E_scaled' if 'E_scaled' in df.columns else 'E'
E_label = 'E (Early Funding, $M)' if E_col == 'E_scaled' else 'E (Early Funding, $)'

# Histogram
ax.hist(df[E_col].dropna(), bins=50, color='red', alpha=0.7, edgecolor='black')

ax.set_xlabel(E_label, fontsize=12, fontweight='bold', color='red')
ax.set_ylabel('Count', fontsize=12)
ax.set_title(f'E Distribution (mean=${df[E_col].mean():.2f}M, std=${df[E_col].std():.2f}M)', 
             fontsize=14, fontweight='bold')

# Add mean and median lines
mean_val = df[E_col].mean()
median_val = df[E_col].median()

ax.axvline(mean_val, color='darkred', linestyle='--', linewidth=2, label=f'Mean: ${mean_val:.2f}M')
ax.axvline(median_val, color='orange', linestyle=':', linewidth=2, label=f'Median: ${median_val:.2f}M')

ax.legend()
ax.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

print(f"\n✅ E distribution plotted")
print(f"   Mean:   ${df[E_col].mean():.2f}M")
print(f"   Median: ${df[E_col].median():.2f}M")
print(f"   Std:    ${df[E_col].std():.2f}M")

if df[E_col].std() > 0:
    print(f"   ✓ PASS: E has variance (can be used in regression)")
else:
    print(f"   ✗ FAIL: E has no variance!")

In [ ]:
# Plot L distribution
fig, ax = plt.subplots(figsize=(8, 5))

l_counts = df['L'].value_counts().sort_index()
ax.bar(l_counts.index, l_counts.values, color=['gray', '#0000FF'], alpha=0.7, edgecolor='black')

ax.set_xlabel('L (Later Stage VC / Series B+)', fontsize=12, fontweight='bold', color='#0000FF')
ax.set_ylabel('Count', fontsize=12)
ax.set_title('L Distribution (2025)', fontsize=14, fontweight='bold')
ax.set_xticks([0, 1])
ax.set_xticklabels(['L=0', 'L=1'])

# Add value labels
for i, (idx, val) in enumerate(l_counts.items()):
    ax.text(idx, val, f'{val:,}\n({val/len(df)*100:.1f}%)', 
            ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

l_rate = l_counts.get(1, 0) / len(df)
print(f"\n✅ L distribution plotted")
print(f"   L=1 rate: {l_rate:.1%}")

if 0.15 <= l_rate <= 0.45:
    print(f"   ✓ PASS: L rate within expected range (15-45%)")
elif l_rate < 0.15:
    print(f"   ⚠️  WARNING: L rate is low (<15%)")
else:
    print(f"   ⚠️  WARNING: L rate is high (>45%)")

In [ ]:
# Plot V distribution
fig, ax = plt.subplots(figsize=(10, 5))

ax.hist(df['V'].dropna(), bins=50, color='green', alpha=0.7, edgecolor='black')

ax.set_xlabel('V (Vagueness)', fontsize=12, fontweight='bold', color='green')
ax.set_ylabel('Count', fontsize=12)
ax.set_title(f'V Distribution (mean={df["V"].mean():.2f}, std={df["V"].std():.2f})', 
             fontsize=14, fontweight='bold')

# Add mean and std lines
ax.axvline(df['V'].mean(), color='darkgreen', linestyle='--', linewidth=2, label=f'Mean: {df["V"].mean():.2f}')
ax.axvline(df['V'].mean() + df['V'].std(), color='orange', linestyle=':', linewidth=2, label=f'±1 Std')
ax.axvline(df['V'].mean() - df['V'].std(), color='orange', linestyle=':', linewidth=2)

ax.legend()
ax.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

print(f"\n✅ V distribution plotted")
print(f"   Mean: {df['V'].mean():.2f}")
print(f"   Std:  {df['V'].std():.2f}")

if df['V'].std() >= 10:
    print(f"   ✓ PASS: Vagueness has reasonable variance (std >= 10)")
elif df['V'].std() >= 5:
    print(f"   ⚠️  WARNING: Vagueness std is small (5-10)")
else:
    print(f"   ✗ FAIL: Vagueness std is too small (<5)")
    print(f"   Run diagnosis: python scripts/diagnose_vagueness_quality.py")

In [ ]:
# Plot F distribution
fig, ax = plt.subplots(figsize=(8, 5))

f_counts = df['F_flexibility'].value_counts().sort_index()
colors = ['gray', 'skyblue']
ax.bar(f_counts.index, f_counts.values, color=colors, alpha=0.7, edgecolor='black')

ax.set_xlabel('F (Flexibility)', fontsize=12, fontweight='bold', color='skyblue')
ax.set_ylabel('Count', fontsize=12)
ax.set_title('F Distribution', fontsize=14, fontweight='bold')
ax.set_xticks([0, 1])
ax.set_xticklabels(['F=0 (Rigid/HW)', 'F=1 (Flexible/SW)'])

# Add value labels
for i, (idx, val) in enumerate(f_counts.items()):
    ax.text(idx, val, f'{val:,}\n({val/len(df)*100:.1f}%)', 
            ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

print(f"\n✅ F distribution plotted")
if len(f_counts) >= 2:
    print(f"   ✓ PASS: F has both values (interaction test possible)")
else:
    print(f"   ✗ FAIL: F has only one value!")
    print(f"   Interaction test will not be possible.")

---
## Stage 4: ⚖️ TEST - Hypothesis Testing & Validation

Test H*: β_VF > 0 (Flexibility amplifies Vagueness effect on Later success)

**Model**: `L ~ z_V * F_flexibility + C(founding_cohort) + C(region)`

**CRITICAL**: NO E control (E is mediator, not confounder)

**Expected**:
- β_V > 0: Vagueness helps Later success (main effect)
- β_VF > 0: Flexibility amplifies the benefit (interaction)

In [ ]:
%%time
# Hypothesis test
print("="*80)
print("HYPOTHESIS TEST: β_VF > 0")
print("="*80)

# Prepare data
analysis_df = df[['L', 'z_V', 'F_flexibility', 'founding_cohort', 'region']].dropna()
print(f"\nAnalysis dataset: {len(analysis_df):,} companies (complete cases)")

# Fit model
formula = "L ~ z_V * F_flexibility + C(founding_cohort) + C(region)"
print(f"\nFormula: {formula}")
print(f"Note: NO E control (E is mediator, not confounder)")

try:
    # Use run_HLVF from models module
    model_hlvf = models.run_HLVF(analysis_df, formula=formula)
    
    print(f"\n✅ Model fitted successfully")
    
    # Display results
    print(f"\n" + "="*80)
    print("COEFFICIENT TABLE")
    print("="*80)
    print(model_hlvf.summary2().tables[1])
    
    # Extract key coefficients
    beta_V = model_hlvf.params.get('z_V', np.nan)
    beta_F = model_hlvf.params.get('F_flexibility', np.nan)
    beta_VF = model_hlvf.params.get('z_V:F_flexibility', np.nan)
    
    p_V = model_hlvf.pvalues.get('z_V', 1.0)
    p_F = model_hlvf.pvalues.get('F_flexibility', 1.0)
    p_VF = model_hlvf.pvalues.get('z_V:F_flexibility', 1.0)
    
    print(f"\n" + "="*80)
    print("KEY RESULTS")
    print("="*80)
    print(f"\nβ_V (main effect):      {beta_V:7.4f}  (p = {p_V:.4f})")
    print(f"β_F (flexibility):      {beta_F:7.4f}  (p = {p_F:.4f})")
    print(f"β_VF (interaction):     {beta_VF:7.4f}  (p = {p_VF:.4f}) ← TARGET")
    
    # One-tailed test for β_VF > 0
    p_one_tailed = p_VF / 2 if beta_VF > 0 else 1 - (p_VF / 2)
    
    print(f"\nOne-tailed test (H1: β_VF > 0):")
    print(f"  p-value (one-tailed): {p_one_tailed:.4f}")
    
    # Verdict
    sig_stars = '***' if p_one_tailed < 0.001 else ('**' if p_one_tailed < 0.01 else ('*' if p_one_tailed < 0.05 else 'ns'))
    
    print(f"\n" + "="*80)
    print("VERDICT")
    print("="*80)
    
    if beta_VF > 0 and p_one_tailed < 0.05:
        print(f"\n✅ REJECT H0: Flexibility AMPLIFIES Vagueness effect")
        print(f"   β_VF = {beta_VF:.4f} > 0, p = {p_one_tailed:.4f} {sig_stars}")
        print(f"\n   Interpretation: High vagueness helps flexible companies MORE")
        print(f"   than rigid companies in achieving later success.")
    else:
        print(f"\n❌ FAIL TO REJECT H0")
        print(f"   β_VF = {beta_VF:.4f}, p = {p_one_tailed:.4f}")
        if beta_VF <= 0:
            print(f"   Reason: β_VF is not positive (wrong direction)")
        else:
            print(f"   Reason: p-value not significant (p >= 0.05)")
    
    # Store for plotting
    hlvf_result = model_hlvf
    
except Exception as e:
    print(f"\n❌ Hypothesis test FAILED: {e}")
    import traceback
    traceback.print_exc()
    hlvf_result = None

In [ ]:
# Run comprehensive validation
print("\n" + "="*80)
print("E/L DESIGN VALIDATION")
print("="*80)
print("\nRun comprehensive validation:")
print("  python scripts/validate_E_L_design.py")
print("\nThis will check:")
print("  ✓ E=1 cohort purity")
print("  ✓ L definition correctness")
print("  ✓ Temporal consistency")
print("  ✓ Cohort stability")
print("  ✓ Impossible transitions")
print("  ✓ Progression rate sanity")

---
## Stage 5: 📈 PLOT2 - Final Interaction Plots

Generate EVF and LVF interaction plots using `plots_F_series.py`.

**EVF Plot** (F1b):
- Shows E ~ V × F interaction (Early funding amount by vagueness and flexibility)
- E is continuous (funding amount), not binary
- F=1 (Flexible/SW): skyblue solid line
- F=0 (Rigid/HW): gray dashed line
- Tests whether flexibility moderates vagueness effect on early funding

**LVF Plot** (F3a):
- Shows L ~ V × F interaction (Later success probability by vagueness and flexibility)
- L is binary (Series B+ achievement)
- F=1 (Flexible/SW): skyblue solid line
- F=0 (Rigid/HW): gray dashed line
- If β_VF > 0, lines should diverge (SW steeper than HW)

In [ ]:
# Generate LVF interaction plot using plots_F_series
if hlvf_result is not None:
    print("Generating LVF interaction plot (F3a)...")
    print("-" * 80)
    
    OUTPUT_DIR = Path("outputs/figures")
    OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
    
    # Generate F3a plot
    paths = plots_F_series.fig_F3a_L_given_F(analysis_df, hlvf_result, OUTPUT_DIR)
    
    print(f"\n✅ LVF plot saved:")
    for fmt, path in paths.items():
        print(f"   {fmt.upper()}: {path}")
    
    # Display inline
    from IPython.display import Image
    display(Image(str(paths['png']), width=800))
else:
    print("❌ Cannot generate LVF plot - model fitting failed")

In [ ]:
# Generate EVF interaction plot (E ~ V × F)
print("Fitting HEV interaction model (E ~ V × F)...")
print("-" * 80)

# Prepare data for EVF
E_col = 'E_scaled' if 'E_scaled' in df.columns else 'E'
evf_df = df[[E_col, 'z_V', 'F_flexibility', 'founding_cohort', 'region']].dropna()
print(f"\nEVF analysis dataset: {len(evf_df):,} companies (complete cases)")

# Fit OLS model with V×F interaction
evf_formula = f"{E_col} ~ z_V * F_flexibility + C(founding_cohort) + C(region)"
print(f"Formula: {evf_formula}")

try:
    import statsmodels.formula.api as smf
    model_hev_interaction = smf.ols(evf_formula, data=evf_df).fit()
    
    print(f"\n✅ HEV interaction model fitted")
    
    # Extract interaction coefficient
    beta_VF_evf = model_hev_interaction.params.get('z_V:F_flexibility', np.nan)
    p_VF_evf = model_hev_interaction.pvalues.get('z_V:F_flexibility', 1.0)
    
    print(f"\nβ_VF (EVF interaction): {beta_VF_evf:.4f} (p = {p_VF_evf:.4f})")
    
    # Generate EVF plot
    OUTPUT_DIR = Path("outputs/figures")
    OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
    
    print(f"\nGenerating EVF interaction plot (F1b)...")
    paths_evf = plots_F_series.fig_F1b_E_given_F(evf_df, model_hev_interaction, OUTPUT_DIR)
    
    print(f"\n✅ EVF plot saved:")
    for fmt, path in paths_evf.items():
        print(f"   {fmt.upper()}: {path}")
    
    # Display inline
    from IPython.display import Image
    display(Image(str(paths_evf['png']), width=800))
    
    hev_interaction_result = model_hev_interaction
    
except Exception as e:
    print(f"❌ EVF plot failed: {e}")
    import traceback
    traceback.print_exc()
    hev_interaction_result = None

In [ ]:
# Generate all F-series plots
print("\nGenerating complete F-series...")
print("-" * 80)

# Prepare results dictionary
results = {
    'HLVF': hlvf_result,
    'HEV_interaction': hev_interaction_result
}

# Generate all plots
try:
    all_paths = plots_F_series.create_F_series(analysis_df, results, OUTPUT_DIR)
    
    print(f"\n✅ F-series plots generated:")
    for fig_name, paths in all_paths.items():
        print(f"\n{fig_name}:")
        for fmt, path in paths.items():
            print(f"   {fmt.upper()}: {path}")
except Exception as e:
    print(f"\n⚠️  Some F-series plots may have failed: {e}")
    import traceback
    traceback.print_exc()

---
## 🎯 Summary

This notebook completed all 5 stages:

1. ✅ **BUILD**: Loaded consolidated E=1 cohort
2. ✅ **DEFINE**: Defined E, L, V, F variables
3. ✅ **PLOT1**: Validated each variable distribution
4. ✅ **TEST**: Tested H*: β_VF > 0 with comprehensive validation
5. ✅ **PLOT2**: Generated LVF interaction plots

### Key Findings

Review the VERDICT section in Stage 4 for hypothesis test results.

### Output Files

- Plots: `outputs/figures/F3a_L_given_F.png` (and `.pdf`)
- All F-series plots in `outputs/figures/`

### Next Steps

1. **If vagueness std is low**: Run `python scripts/diagnose_vagueness_quality.py`
2. **For full validation**: Run `python scripts/validate_E_L_design.py`
3. **For industry subsets**: Run with quantum or transportation cohorts

---